# 0. Import Required Libraries

In [1]:
import mlflow
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

import math
from lib.utils import *

# 1. Define Classifier Architecture

In [2]:
class Classifier(nn.Module):
    def __init__(self, channel, in_len, out_len):
        super(Classifier, self).__init__()
        
        self.fc1_size = in_len ** 2
        self.fc2_size = self.fc1_size * 2
        self.fc3_size = self.fc2_size
        
        self.fc1 = nn.Linear(self.fc1_size, self.fc2_size)
        self.fc2 = nn.Linear(self.fc2_size, self.fc3_size)
        self.fc3 = nn.Linear(self.fc3_size, out_len)
        
        self.dropout = nn.Dropout(p=0.4)
    def forward(self, x):
        # Flatten input
        x = x.view(x.shape[0],x.shape[1], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        
        x = self.dropout(F.relu(self.fc2(x)))
        
        x = torch.sigmoid(self.fc3(x))
        return x

In [3]:
def chunk_image(image):
    """
    Break a multispectral image into 9x4x4 tensors
    image: Pytorch tensor with dimensions CxHxW where in this application
        C is channels, H is height, and W is width
    returns:
        image: numpy array representing 
        ground_truth: 
    """
    data = []

    height = image[0].shape[0]
    width = image[0].shape[1]
    
    for i in range(0, height, 4):
        band_chunks = []

        if i + 3 > height - 1:
            break

        for j in range(0, width, 4):
            rows = []
            
            if j + 4 > width:
                break
                
            for band in image:
                chunk = np.ndarray(shape=(0))
                #try:
                half1 = np.append([band[i + 0][j:j + 4]], [band[i + 1][j:j + 4]], axis=0)
                half2 = np.append([band[i + 2][j:j + 4]], [band[i + 3][j:j + 4]], axis=0)
                #except:
                #print(height, width)
                #print(i, j)
                #break
                chunk = np.append(half1, half2, axis=0)

                rows.append(chunk)

            band_chunks.append(rows)

        data.append(band_chunks)
        
    data = np.array(data)
    return data

In [4]:
model = Classifier(9, 2, 16)

In [5]:
data, filenames = load_data(10, "./data/modis")

In [6]:
labels = [data[i][0] for i in range(len(data))]
train_data = [data[i][1:] for i in range(len(data))]

In [7]:
print("{:30} shape: (batch, channel, height, width)".format("filename"))

for index, image in enumerate(train_data):
    chunked_image = np.reshape(chunk_image(image), (-1, 9, 4, 4))
    print("{:30} shape: {}".format(filenames[index], chunked_image.shape))

filename                       shape: (batch, channel, height, width)
arkansas_city.tif              shape: (34970, 9, 4, 4)
assiniboine.tif                shape: (136406, 9, 4, 4)
bay_area.tif                   shape: (197788, 9, 4, 4)
berkeley.tif                   shape: (26880, 9, 4, 4)
kashmore.tif                   shape: (206276, 9, 4, 4)
kashmore_north.tif             shape: (20572, 9, 4, 4)
katrina.tif                    shape: (79158, 9, 4, 4)
katrina_slidell.tif            shape: (15730, 9, 4, 4)
malawi.tif                     shape: (26520, 9, 4, 4)
mississippi_june.tif           shape: (51675, 9, 4, 4)
mississippi_may.tif            shape: (51675, 9, 4, 4)
parana.tif                     shape: (42920, 9, 4, 4)
sava.tif                       shape: (47040, 9, 4, 4)
sava_west.tif                  shape: (25608, 9, 4, 4)
unflooded_mississippi.tif      shape: (104145, 9, 4, 4)
unflooded_new_orleans.tif      shape: (88006, 9, 4, 4)


In [8]:
model.forward(torch.randn(32, 9, 2, 2)).shape

torch.Size([32, 9, 16])